<a href="https://colab.research.google.com/github/handsome1201/Failure/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bar graph & Circle graph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyyaml==5.4.1
!pip install extcolors
!pip install easyocr

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import cv2
import numpy as np
from PIL import Image
import easyocr
import xml.etree.ElementTree as ET
import extcolors
import os
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyyaml: filename=PyYAML-5.4.1-cp310-cp310-linux_x86_64.whl size=45658 sha256=45d5aeb5aba09cbb22b595b3124167fc9a627b84a66c0073aff35595c3d2a4c3
  Stored in directory: /root/.cache/pip/wheels/c7/0d/22/696ee92245ad710f506eee79bb05c740d8abccd3ecdb778683
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.1
    Uninstalling PyYAML-5.1:
      Successfully uninstalled PyYAML-5.1


### 그래프 종류 판별

In [ ]:
def wrong_detect_delete(data):
  count=0
  number=re.compile('[\d]')
  out_str=''
  for j in data :
      j = list(j)
      for alphabet in j:
          if number.match(alphabet)!=None:
              out_str+=alphabet
          elif alphabet==',' or alphabet=='.':
              out_str+=alphabet

  if data[-1]!="%" and len(out_str)>5 and (out_str[len(out_str)-2:len(out_str)] =='96') | (out_str[len(out_str)-2:len(out_str)] =='95') | (out_str[len(out_str)-2:len(out_str)] =='36'):

      out_str = list(out_str)

      out_str[len(out_str)-2:len(out_str)] = ''
      out_str=''.join(out_str)
  return out_str

In [ ]:
def Vertical(image_path):

  if torch.cuda.is_available():
      reader = easyocr.Reader(['en', 'ko'], gpu=True)
  else:
      reader = easyocr.Reader(['en', 'ko'], gpu=False)

  result = reader.readtext(image_path)

  img = Image.open(image_path)
  img_array = np.array(img)
  gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
  result = reader.readtext(gray)

  grouped_values = {}

  for i in range(len(result)):
      text = wrong_detect_delete(result[i][1])

      # 그림 단어가 포함되어 있는 경우
      if '그림' in text:
          grouped_values['제목'] = text
      else:
          x, y = min(list(_[0] for _ in result[i][0])), min(list(_[1] for _ in result[i][0]))

          # 값의 차이가 25 이하인 값을 찾아서 묶어줍니다
          for j in range(i - 1, -1, -1):
              if abs(min(list(_[0] for _ in result[j][0])) - x) <= 25:
                  if y < min(list(_[1] for _ in result[j][0])):
                      grouped_values[text] = result[j][1]
                  else:
                      grouped_values[result[j][1]] = text
                  break

  # Vertical 함수 호출 및 결과 출력
  grouped_values = Vertical(image_path)
  for key, value in grouped_values.items():
      grouped_values[key] = wrong_detect_delete(value)

  return grouped_values

In [ ]:
def Horizontal(image_path):

  if torch.cuda.is_available():
      reader = easyocr.Reader(['en', 'ko'], gpu=True)
  else:
      reader = easyocr.Reader(['en', 'ko'], gpu=False)

  result = reader.readtext(image_path)

  img = Image.open(image_path)
  img_array = np.array(img)
  gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
  result = reader.readtext(gray)

  grouped_values = {}

  # 그림으로 시작하는 단어가 있을 경우 '제목'으로 key값 설정
  for i in range(len(result)):
      if result[i][1].startswith('그림'):
          grouped_values['제목'] = result[i][1]
          break

  for i in range(len(result)):
      text = result[i][1]
      x, y = min(list(_[0] for _ in result[i][0])), min(list(_[1] for _ in result[i][0]))

      # 값의 차이가 25 이하인 값을 찾아서 묶어줍니다
      for j in range(i - 1, -1, -1):
          if abs(min(list(_[1] for _ in result[j][0])) - y) <= 25:
              if x < min(list(_[0] for _ in result[j][0])):
                  grouped_values[text] = result[j][1]
              else:
                  grouped_values[result[j][1]] = text
              break

  # Vertical 함수 호출 및 결과 출력
  grouped_values = Vertical(image_path)
  for key, value in grouped_values.items():
      grouped_values[key] = wrong_detect_delete(value)


  return grouped_values

In [ ]:
def circle(image_path):

  def compare_rgb_tuple(rgb1, rgb2):  # 두 rgb 값들이 유사한지를 판단해 boolean으로 반환하는 함수
      root = ET.Element("root")
      r_diff_abs = abs(rgb1[0] - rgb2[0])
      g_diff_abs = abs(rgb1[1] - rgb2[1])
      b_diff_abs = abs(rgb1[2] - rgb2[2])
      tot_abs = r_diff_abs + g_diff_abs + b_diff_abs
      if r_diff_abs<30 and g_diff_abs<30 and b_diff_abs<30 and tot_abs<70 :
          return True
      return False

  img = Image.open(image_path)  # 이미지 열기
  rgb_img = img.convert("RGB")

  reader = easyocr.Reader(['ko','en'], gpu=True)    # ocr로 이미지 읽기
  result1 = reader.readtext(image_path)

  num = len(result1)
  text_list=[]  # ocr로 읽은 text list
  color_list=[] # text가 위치한 곳의 color list
  grouped_values = {}
  title_loc=0
  istitle=0

  for i in range(num):          #제목이 있는 경우 제목을 분리
    if len(result1[i][1])>2:
        if '그림' in result1[i][1]:
            grouped_values['제목']=result1[i][1]
            title_loc=i
            istitle=1

  if istitle==1:
    result1.remove(result1[title_loc])
  num = len(result1)

  for i in range(num):  # text가 위치한 곳의 rgb 값을 읽어옴.
      min_x=result1[i][0][0][0]
      min_y=result1[i][0][0][1]
      width=0
      height=0
      text_list.append(result1[i][1])

      for j in range(3):
          j=j+1
          if min_x>=result1[i][0][j][0]:
              min_x=result1[i][0][j][0]
          else :
              width = result1[i][0][j][0]-min_x
          if min_y>=result1[i][0][j][1]:
              min_y=result1[i][0][j][1]
          else :
              height = result1[i][0][j][1]-min_y
      crop_img = rgb_img.crop((min_x, min_y, min_x+width, min_y+height))
      crop_img.save('tb_img_ext'+str(i)+'.png', 'png')
      saved_croped_img = Image.open('tb_img_ext'+str(i)+'.png')
      colors, _ = extcolors.extract_from_image(saved_croped_img, limit=1)
      for c in colors:
          color_list.append(c[0])
      os.remove('tb_img_ext'+str(i)+'.png')

  rgb_group=[]
  group=[]
  arr_count=0
  filled_group=0

  for rgb_val in color_list:    # text의 rgb 값이 겹치는 것끼리 group화
      if len(rgb_group)==0 :
          rgb_group.append(rgb_val)
          group.append([])
          group[filled_group].append(text_list[arr_count])
          filled_group+=1

      else :
          is_old=0
          group_count=0
          for selected_rgb in rgb_group:
              if compare_rgb_tuple(selected_rgb, rgb_val):
                  group[group_count].append(text_list[arr_count])
                  is_old=1
                  break

              group_count+=1
          if is_old==0:
              rgb_group.append(rgb_val)
              group.append([])
              group[filled_group].append(text_list[arr_count])
              filled_group+=1
      arr_count+=1
  merged_group=[]
  count=0

  for i in group :      # group화된 text들을 배경색을 기준으로 나누어 저장
      out_str=''
      merged_group.append([[],[]])

      for j in i:
          if (j[len(j)-2:len(j)] =='96') | (j[len(j)-2:len(j)] =='95') | (j[len(j)-2:len(j)] =='36'):
              j = list(j)
              j[len(j)-2:len(j)] = ''
              ''.join(j)
              out_percent=''
              for k in j:
                  out_percent+=str(k)
              merged_group[count][1].append(out_percent)
              check=1
          else :
              out_str+=j

      merged_group[count][0].append(out_str)
      count+=1

  number=re.compile('[\d]')

  for i in merged_group:            # group 내의 항목과 항목값을 분류
      out_str1=''
      out_num1=''
      out_str2=''
      out_num2=''

      for j in i[0]:
          j=list(j)
          for alphabet in j:
            if number.match(alphabet)!=None:
                out_num1+=alphabet
            elif alphabet==',' or alphabet=='.' or alphabet=='%':
                out_num1+=alphabet
            else:
                out_str1+=alphabet

      for j in i[1]:
          j=list(j)
          for alphabet in j:
            if number.match(alphabet)!=None:
                out_num2+=alphabet
            elif alphabet==',' or alphabet=='.'or alphabet=='%':
                out_num2+=alphabet
            else:
                out_str2+=alphabet

      if out_num1!='' and out_str1!='':

          if out_num2!='' and out_str2!='':
            grouped_values[out_str1]=out_num1
            grouped_values[out_str2]=out_num2

          elif out_num2=='' and out_str2!='':
            grouped_values[out_str1+out_str2]=out_num1
          else:
            grouped_values[out_str1]=out_num1

      elif out_num2!='' and out_str2!='':
          if out_num1!='' and out_str1!='':
            grouped_values[out_str1]=out_num1
            grouped_values[out_str2]=out_num2

          elif out_num1=='' and out_str1!='':
            grouped_values[out_str1+out_str2]=out_num2

          else:
            grouped_values[out_str2]=out_num2

      elif out_str1!='' and out_num2!='':
          grouped_values[out_str1]=out_num2
      elif out_str2!='' and out_num1!='':
          grouped_values[out_str2]=out_num1

  return grouped_values

In [ ]:
def detect_graph_orientation(image_path):
    class Net(nn.Module):
        def __init__(self, num_classes):
            super(Net, self).__init__()
            self.conv = nn.Sequential(
                nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
                nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.fc = nn.Sequential(
                nn.Linear(32 * 56 * 56, 128),
                nn.ReLU(),
                nn.Linear(128, num_classes)
            )

        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)
            x = self.fc(x)
            return x

    # 새로운 파일에서 사용할 모델 정의 및 초기화
    num_classes = 2  # 이전에 학습한 모델의 클래스 개수에 맞게 설정
    model = Net(num_classes)
    state_dict = torch.load("/content/drive/MyDrive/자료/model.pth", map_location=torch.device('cpu'))
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in state_dict.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict, strict=False)
    class_names = ["horizontal", "vertical"]
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # 이미지 크기 조정
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    outputs = model(image)
    _, predicted = torch.max(outputs.data, 1)
    predicted_class = class_names[predicted.item()]

    if predicted_class == 'horizontal':
        return 2
    elif predicted_class == 'vertical':
        return 1


In [ ]:
def detect_circle_orientation(image_path):

    src = cv2.imread(image_path)
    dst = src.copy()
    gray1 = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray1, (0, 0), 1)
    height, width = gray.shape
    gray_len = min(height, width)

    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT, 1, 50, param1=300, param2=0.8, minRadius=int(gray_len / 6),
                               maxRadius=int(gray_len / 2))
    if circles is not None:
        return 3

### input image

In [ ]:
image_path = '/content/drive/MyDrive/자료/train/vertical/수직 제목.png'

if detect_circle_orientation(image_path) == 3:
  circle(image_path)
elif detect_graph_orientation(image_path) == 1:
  Vertical(image_path)
else:
  Horizontal(image_path)

OutOfMemoryError: ignored

In [ ]:
if torch.cuda.is_available():
    reader = easyocr.Reader(['en', 'ko'], gpu=True)
else:
    reader = easyocr.Reader(['en', 'ko'], gpu=False)

result = reader.readtext(image_path)

img = Image.open(image_path)
img_array = np.array(img)
gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
result = reader.readtext(gray)

grouped_values = {}

for i in range(len(result)):
    text = wrong_detect_delete(result[i][1])

    # 그림 단어가 포함되어 있는 경우
    if '그림' in text:
        grouped_values['제목'] = text
    else:
        x, y = min(list(_[0] for _ in result[i][0])), min(list(_[1] for _ in result[i][0]))

        # 값의 차이가 25 이하인 값을 찾아서 묶어줍니다
        for j in range(i - 1, -1, -1):
            if abs(min(list(_[0] for _ in result[j][0])) - x) <= 25:
                if y < min(list(_[1] for _ in result[j][0])):
                    grouped_values[text] = result[j][1]
                else:
                    grouped_values[result[j][1]] = text
                break

# Vertical 함수 호출 및 결과 출력
grouped_values = Vertical(image_path)
for key, value in grouped_values.items():
    grouped_values[key] = wrong_detect_delete(value)


In [ ]:
grouped_values